# Proyecto integrador (2022)
Autor: Eric S. Tellez <eric.tellez@infotec.mx> <br/>

A lo largo del curso de Recuperación de Información se vieron diferentes maneras de modelar texto, tomando encuenta su vocabulario o su semántica. En particular, se manipulo de diferentes formas el vocabulario para resaltar o disminuir efectos que impactarán en la calidad de respuesta o la velocidad de los resultados. Se utilizó el índice invertido para representaciones dispersas y búsqueda métrica con vectores densos, dando diferentes posibilidades para cumplir con los diferentes requerimientos de los sistemas de información.

Adicional a la búsqueda, también se reviso la parte de organización por grupos, _clustering_, y visualización utilizando técnicas de reducción de dimensión no-lineal. 

Se espera que todos estos temas sean de utilidad en las actividades de un científico de datos, tanto en la etapa de análisis exploratorio de los datos como en la construcción de sistemas inteligentes.

# Actividades

- Construcción de un mini-sistema de información que solucione un problema que usted conozca
- Recolección de documentos
- Modelado de los documentos
- Indexamiento, búsqueda y presentación de documentos
- Análisis y visualización de datos
- Reporte


# Ejemplo: Paquetes en el registro principal del lenguaje Julia.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
Pkg.status()

Status `~/Cursos/IR-2024/Unidades/Project.toml`
  [87b137f8] BagOfWords v0.3.3
  [336ed68f] CSV v0.10.12
  [aaaa29a8] Clustering v0.15.6
  [944b1d66] CodecZlib v0.7.3
  [a80b9123] CommonMark v0.8.12
  [a93c6f00] DataFrames v1.6.1
  [c5bfea45] Embeddings v0.4.5 `~/.julia/dev/Embeddings`
  [ac1192a8] HypertextLiteral v0.9.5
  [7073ff75] IJulia v1.24.2
  [c601a237] Interact v0.10.5
  [b20bd276] InvertedFiles v0.7.1
  [033835bb] JLD2 v0.4.41
  [682c06a0] JSON v0.21.4
  [4dca28ae] KNearestCenters v0.7.7
  [8ef0a80b] Languages v0.4.6
  [eb30cadb] MLDatasets v0.7.14
  [06eb3307] ManifoldLearning v0.9.0
⌅ [6f286f6a] MultivariateStats v0.9.0
  [91a5bcdd] Plots v1.39.0
  [438e738f] PyCall v1.96.4
  [ca7ab67e] SimSearchManifoldLearning v0.2.10
  [053f045d] SimilaritySearch v0.11.1
  [fb8f903a] Snowball v0.1.1
⌅ [2913bbd2] StatsBase v0.33.21
  [f3b207a7] StatsPlots v0.15.6
  [24678dba] TSne v1.3.0
  [7f6f6c8a] TextSearch v0.18.0
  [0f1e0344] WebIO v0.8.21
  [c64b6f0f] Word2Vec v0.5.3
⌅ [a5390f91] 

In [3]:
using SimilaritySearch, Interact, SimSearchManifoldLearning, TextSearch, StatsBase, Clustering, ZipFile, CommonMark, JSON, Base64, Plots, LinearAlgebra, HypertextLiteral
include("read_datasets.jl")

WebIO._IJuliaInit()

get_julia_packages (generic function with 1 method)

In [4]:
function sections(readmetext, maxsections=1)
    S = []
    for p in eachmatch(r"#(.+?)\n([^#]+)"ims, readmetext)
        if length(p.captures) == 2
            head, text = p.captures
            head = strip(replace(head, "#" => ""))
            push!(S, (; head, text))
            length(S) == maxsections && break
        end
    end
    
    if 0 == length(S)
        push!(S, (head="", text=readmetext))
    end
    S
end

function packages_metadata()
    packages = ZipFile.Reader(get_julia_packages())
    readme = Dict{String, Int}()
    metadata = Dict{String, Int}()
    
    for (i, file) in enumerate(packages.files)
        arr = splitpath(file.name)
        name, kind = arr[end-1], arr[end]
        if kind == "Metadata.json"
            metadata[name] = i
        else
            readme[name] = i
        end
    end

    packages, readme, metadata
end

function read_zipped(z)
    seekstart(z)
    JSON.parse(read(z, String))
end

#metadata(name::String) = read_zipped(name, D.metadata)

function readme(z)
    r = read_zipped(z)
    String(base64decode(r["content"]))
end

# readme(name::String) = readme(D.readme[name])

function create_dataset()
    packages, readme_, metadata_ = packages_metadata()
    
    names = String[]
    urls = String[]
    descriptions = String[]
    stars = Int[]
    corpus = String[]
    zipid = typeof((; readme=1, metadata=1))[]
    name2id = Dict{String,Int}()
    
    for (k, i) in readme_
        s = only(sections(readme(packages.files[i]), 1))
        j = metadata_[k]
        m = read_zipped(packages.files[j])
        push!(names, k)
        push!(urls, m["html_url"])
        d = m["description"]
        push!(descriptions, d === nothing ? "_no description_" : d)
        push!(stars, m["watchers_count"])
        push!(corpus, s.text)
        push!(zipid, (readme=i, metadata=j))
        name2id[k] = length(names)
    end    
    
    (; names, urls, descriptions, stars, zipid, name2id, corpus, packages)
end

function readme_and_metadata(name::String, D::NamedTuple)
    f = D.zipid[D.name2id["SimilaritySearch"]]
    readme(D.packages.files[f.readme]), read_zipped(D.packages.files[f.metadata])
end

function package(name::String, D::NamedTuple)
    id = D.name2id[name]
    (; id, name, zipid=D.zipid[id], text=D.corpus[id])
end

package (generic function with 1 method)

In [5]:
D = create_dataset();

In [6]:
function create_index(vectors)
    dist = NormalizedCosineDistance()
    db = VectorDatabase(vectors)
    index = SearchGraph(; dist, db)
    minrecall = MinRecall(0.9)
    ctx = SearchGraphContext(hyperparameters_callback=OptimizeParameters(minrecall))
    index!(index, ctx)
    optimize_index!(index, ctx, minrecall)
    index
end

function text_model_and_vectors(corpus;
        textconfig = TextConfig(group_usr=false, group_url=true, del_diac=true, del_punc=true, lc=true, group_num=true, nlist=[], qlist=[4]),
        voc = Vocabulary(textconfig, corpus),
        model = VectorModel(IdfWeighting(), TfWeighting(), voc)
    )
    model = filter_tokens(model) do t
        5 ≤ t.ndocs ≤ 1000
    end
    vectors = vectorize_corpus(model, corpus)
    (; textconfig, model, vectors)
end

myvectorize(text::String, T::NamedTuple) = vectorize(T.model, text)
myvectorize_corpus(corpus, T::NamedTuple) = vectorize_corpus(T.model, corpus)

myvectorize_corpus (generic function with 1 method)

In [7]:
T = text_model_and_vectors(D.corpus)
@show T.model
@time index = create_index(T.vectors);

T.model = {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 21210
    trainsize=6686
    maxoccs=167309                                    
}
 19.421375 seconds (4.44 M allocations: 406.315 MiB, 1.11% gc time, 23.63% compilation time)


In [8]:
function search_and_display(text, k, D, T)
    res = KnnResult(k)
    search(index, myvectorize(text, T), res)
    display("text/markdown", "# Results for `$text`")
    
    for (i, p) in enumerate(res)
        display(@htl """
            <hr />
            <div style="padding: 0.5em; border-style: solid; border-color: #557799;">
            <div><a href="$(D.urls[p.id])">$(D.names[p.id])</a> &nbsp;&nbsp; ⭐ $(D.stars[p.id]) </div>
            <div> $(D.descriptions[p.id])</div>
            <span style="color: #aaaa30;">debug:: i: $(i), id: $(p.id), dist=$(p.weight)</span>
            </div>
        """)
        display("text/markdown", D.corpus[p.id])
    end
end

#search_and_display("similarity search nearest neighbor", 10, D, T)


search_and_display (generic function with 1 method)

In [9]:
search_and_display("similarity search", 10, D, T)

# Results for `similarity search`

<hr />
    <div style="padding: 0.5em; border-style: solid; border-color: #557799;">
    <div><a href="https://github.com/kernelmethod/LSHFunctions.jl">LSHFunctions</a> &nbsp;&nbsp; ⭐ 12 </div>
    <div> Locality-sensitive hashing (LSH) in Julia.</div>
    <span style="color: #aaaa30;">debug:: i: 1, id: 2843, dist=0.49194074</span>
    </div>


[![Stable docs](https://img.shields.io/badge/docs-stable-blue.svg)](https://kernelmethod.github.io/LSHFunctions.jl/stable/) [![Dev docs](https://img.shields.io/badge/docs-dev-blue.svg)](https://kernelmethod.github.io/LSHFunctions.jl/dev/)
[![Build Status](https://github.com/kernelmethod/LSHFunctions.jl/workflows/CI/badge.svg)](https://github.com/kernelmethod/LSHFunctions.jl/actions)
[![Codecov](https://codecov.io/gh/kernelmethod/LSHFunctions.jl/branch/master/graph/badge.svg)](https://codecov.io/gh/kernelmethod/LSHFunctions.jl)
[![DOI](https://zenodo.org/badge/197700982.svg)](https://zenodo.org/badge/latestdoi/197700982)

A Julia package for [locality-sensitive
hashing](https://en.wikipedia.org/wiki/Locality-sensitive_hashing) to accelerate
similarity search.

- [What's LSH?](

<hr />
    <div style="padding: 0.5em; border-style: solid; border-color: #557799;">
    <div><a href="https://github.com/zsz00/Faiss.jl">Faiss</a> &nbsp;&nbsp; ⭐ 3 </div>
    <div> Julia wrapper around the Faiss library for similarity search with PythonCall.jl</div>
    <span style="color: #aaaa30;">debug:: i: 2, id: 3388, dist=0.5954186</span>
    </div>

 [![][docs-dev-img]][docs-dev-url]
 
A simple Julia wrapper around the [Faiss](https://github.com/facebookresearch/Faiss) library for similarity search with [`PythonCall.jl`](https://github.com/cjdoris/PythonCall.jl).

While functional and faster than [`NearestNeighbors.jl`](https://github.com/KristofferC/NearestNeighbors.jl).

Faiss is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Faiss is written in C++ with complete wrappers for Python/numpy. Some of the most useful algorithms are implemented on the GPU. It is developed primarily at Facebook AI Research.




...


[![Stable](https://img.shields.io/badge/docs-stable-blue.svg)](https://johnnychen94.github.io/BlockMatching.jl/stable)
[![Dev](https://img.shields.io/badge/docs-dev-blue.svg)](https://johnnychen94.github.io/BlockMatching.jl/dev)
[![Build Status](https://github.com/johnnychen94/BlockMatching.jl/workflows/CI/badge.svg)](https://github.com/johnnychen94/BlockMatching.jl/actions)
[![Coverage](https://codecov.io/gh/johnnychen94/BlockMatching.jl/branch/master/graph/badge.svg)](https://codecov.io/gh/johnnychen94/BlockMatching.jl)

`BlockMatching` aims to provide a sophisticated implementation on common [block matching
algorithms](https://en.wikipedia.org/wiki/Block-matching_algorithm) for image processing and
computer vision tasks. Block matching is a data and computational intense algorithm, performance is
of high priority for this package.

🚧 This is still a WIP project.

Two functions are provided as the standard API:

- `best_match`: finds the best matching candidate. This is also known as nearest neighbor search.
- `multi_match`: sort the similarities of all candidates and return the smallest K results. This is sometimes known as K nearest neighbor search or top-k selection.

Available block matching strategies:

- `FullSearch`(brute force): search among all possible candidates. This gives the most accurate result 
  but is computationally intensive. CUDA is supported for commonly used distances defined in 
  [Distances.jl].


[Distances.jl]: https://github.com/JuliaStats/Distances.jl


<hr />
    <div style="padding: 0.5em; border-style: solid; border-color: #557799;">
    <div><a href="https://github.com/Kolaru/BranchAndPrune.jl">BranchAndPrune</a> &nbsp;&nbsp; ⭐ 9 </div>
    <div> Branch and prune interface for Julia</div>
    <span style="color: #aaaa30;">debug:: i: 9, id: 2434, dist=0.8274313</span>
    </div>


This package aims at providing an interface for branch and prune search in Julia.



<hr />
    <div style="padding: 0.5em; border-style: solid; border-color: #557799;">
    <div><a href="https://github.com/lotcher/ElasticSearch.jl">ElasticSearch</a> &nbsp;&nbsp; ⭐ 3 </div>
    <div> A ElasticSearch client for Julia</div>
    <span style="color: #aaaa30;">debug:: i: 10, id: 300, dist=0.8288296</span>
    </div>

*A ElasticSearch client for Julia*

